In [2]:
import cnot_network

from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sbs
import numpy as np
sbs.set_context("paper")
sbs.set_theme()
sbs.set_style("dark")

from surface_code_routing.tikz_utils import tikz, tex

sbs.set_style("whitegrid", {'grid.linestyle': '-.'})

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
})

def plt_data(condition, x_axis, y_axis, target=None):
    data = filter(lambda x: x[0] == target, zip(condition, x_axis, y_axis))
    x = []
    y = []
    for _, x_val, y_val in data:
        x.append(x_val)
        y.append(y_val)
    return {'x':x, 'y':y}   

In [3]:
from functools import partial
from surface_code_routing.lib_instructions import T_Factory, Toffoli, T_gate

toff_height = 14
toff_width = toff_height

qcb_size = 20

t_factory_l1 = cnot_network.T_Factory()
toffoli_gate = cnot_network.Toffoli

t_factory_l2 = T_Factory(t_factory_l1, height=8, width=10, t_gate=T_gate(t_factory_l1))
t_gate_l2 = T_gate(factory=t_factory_l2)

t_factory_l3 = T_Factory(t_factory_l2, height=11, width=12, t_gate=T_gate(t_factory_l2))
t_gate_l3 = T_gate(factory=t_factory_l3)

extern_toffoli_l1 = cnot_network.toffoli(toff_height, toff_width, t_factory=t_factory_l1)
extern_toffoli_l2 = cnot_network.toffoli(toff_height, toff_width, t_factory=t_factory_l2)
extern_toffoli_l3 = cnot_network.toffoli(toff_height, toff_width, t_factory=t_factory_l3)

# Raw T injection sites
ccz_factory_l1 = cnot_network.CCZ_factory(toff_height, toff_width, t_factory=None)
ccz_factory_l2 = cnot_network.CCZ_factory(toff_height, toff_width, t_factory=t_factory_l1)
ccz_factory_l3 = cnot_network.CCZ_factory(toff_height, toff_width, t_factory=t_factory_l2)

t_factories = (t_factory_l1, t_factory_l2, t_factory_l3)
extern_toffolis = (extern_toffoli_l1, extern_toffoli_l2, extern_toffoli_l3)
ccz_factories = (ccz_factory_l1, ccz_factory_l2, ccz_factory_l3)

In [4]:
from functools import partial
from surface_code_routing.lib_instructions import T_Factory, Toffoli, T_gate

toff_height = 14
toff_width = toff_height

qcb_size = 20

t_factory_l1 = cnot_network.T_Factory()
toffoli_gate = cnot_network.Toffoli

t_factory_l2 = T_Factory(t_factory_l1, height=8, width=10, t_gate=T_gate(t_factory_l1))
t_gate_l2 = T_gate(factory=t_factory_l2)

t_factory_l3 = T_Factory(t_factory_l2, height=11, width=12, t_gate=T_gate(t_factory_l2))
t_gate_l3 = T_gate(factory=t_factory_l3)

extern_toffoli_l1 = cnot_network.toffoli(toff_height, toff_width, t_factory=t_factory_l1)
extern_toffoli_l2 = cnot_network.toffoli(toff_height, toff_width, t_factory=t_factory_l2)
extern_toffoli_l3 = cnot_network.toffoli(toff_height, toff_width, t_factory=t_factory_l3)

# Raw T injection sites
ccz_factory_l1 = cnot_network.CCZ_factory(6, 4, t_factory=None)
ccz_factory_l2 = cnot_network.CCZ_factory(9, 7, t_factory=t_factory_l1)
ccz_factory_l3 = cnot_network.CCZ_factory(11, 11, t_factory=t_factory_l2)

t_factories = (t_factory_l1, t_factory_l2, t_factory_l3)
extern_toffolis = (extern_toffoli_l1, extern_toffoli_l2, extern_toffoli_l3)
ccz_factories = (ccz_factory_l1, ccz_factory_l2, ccz_factory_l3)

In [5]:
import numpy as np
from functools import partial
from surface_code_routing.dag import DAG
from surface_code_routing.symbol import Symbol
from surface_code_routing.instructions import INIT, CNOT, MEAS, X, Hadamard, X_MULTI
from surface_code_routing.lib_instructions import T_Factory, Toffoli, T_gate

from surface_code_routing.compiled_qcb import compile_qcb

low = 2
high = 10
mct = partial(range, low, high + 1)

def multi_toffoli(n_ctrls, height, width, *externs, init=True, toffoli=Toffoli, **kwargs):

    dag = DAG(f'MCToff_{n_ctrls}')
    
    if init:
        # Initialise gates
        for i in range(n_ctrls):
            dag.add_gate(INIT(f'ctrl_{i}'), )

        for i in range(1, n_ctrls - 1):
            dag.add_gate(INIT(f'anc_{i}'), )

        dag.add_gate(INIT(f'targ'), )

    if n_ctrls == 2:
        dag.add_gate(toffoli('ctrl_0', 'ctrl_1', 'targ'))
        return compile_qcb(dag, height, width, *externs, **kwargs) 
    
    # First Toffoli
    dag.add_gate(toffoli(
        *list(map(dag.__getitem__, 
        ('ctrl_0', 'ctrl_1', 'anc_1')
           ))
    ))

    for i in range(2, n_ctrls - 1):
        dag.add_gate(toffoli(f'anc_{i - 1}', f'ctrl_{i}', f'anc_{i}'))
        
        
    dag.add_gate(toffoli(f'anc_{n_ctrls - 2}', f'ctrl_{n_ctrls - 1}', 'targ'))

    for i in range(n_ctrls - 2,  1, -1):
        dag.add_gate(toffoli(f'anc_{i - 1}', f'ctrl_{i}', f'anc_{i}'))

    dag.add_gate(toffoli('ctrl_0', 'ctrl_1', 'anc_1'))

    return compile_qcb(dag, height, width, *externs, **kwargs) 

In [5]:
results = {'distillation':[], 'n_ctrl': [], 'height':[], 'runtime':[], 'volume':[], 'delay':[]}

for distillation, t_factory in zip((1, 2, 3), t_factories):
    t_gate = T_gate(factory=t_factory)
    toffoli = partial(Toffoli, T=t_gate)
    for i in mct():
        print(f"\rCompiling: {distillation} : {i}" + " " * 5, end='', flush=True)
        qcb = multi_toffoli(i, qcb_size, qcb_size, t_factory, toffoli=toffoli, allocator_kwargs={'over_allocate':True})
        results['distillation'].append(distillation)
        results['n_ctrl'].append(i)
        results['height'].append(qcb_size)
        results['runtime'].append(qcb.n_cycles())
        results['volume'].append(qcb.space_time_volume())
        results['delay'].append(qcb.delays())

results_dag = results

Compiling: 3 : 10     

In [6]:
results = {'distillation':[], 'n_ctrl': [], 'height':[], 'runtime':[], 'volume':[], 'delay':[]}
repeats = 5
for distillation, extern_toffoli in zip((1, 2, 3), extern_toffolis):
    extern_toffoli_gate = cnot_network.toff_extern(extern_toffoli)
    for i in mct():
        for j in range(repeats):
            print(f"\rCompiling: {distillation} : {i}" + " " * 5, end='', flush=True)
            qcb = multi_toffoli(i, qcb_size, qcb_size, extern_toffoli, toffoli=extern_toffoli_gate, allocator_kwargs={'over_allocate':False})
            results['distillation'].append(distillation)
            results['n_ctrl'].append(i)
            results['height'].append(qcb_size)
            results['runtime'].append(qcb.n_cycles())
            results['volume'].append(qcb.space_time_volume())
            results['delay'].append(qcb.delays())

results_extern = results

Compiling: 3 : 10     

In [7]:
results = {'distillation':[], 'n_ctrl': [], 'height':[], 'runtime':[], 'volume':[], 'delay':[]}
repeats = 5

for distillation, ccz_factory in zip((1, 2, 3), ccz_factories):
    ccz_toffoli_gate = partial(cnot_network.toffoli_from_ccz, ccz_factory=ccz_factory)
    
    for i in mct():
        for j in range(repeats):
            print(f"\rCompiling: {distillation} : {i} {j}" + " " * 5, end='', flush=True)
            qcb = multi_toffoli(i, qcb_size, qcb_size, ccz_factory, toffoli=ccz_toffoli_gate, allocator_kwargs={'over_allocate': True})
            results['distillation'].append(distillation)
            results['n_ctrl'].append(i)
            results['height'].append(qcb_size)
            results['runtime'].append(qcb.n_cycles())
            results['volume'].append(qcb.space_time_volume())
            results['delay'].append(qcb.delays())

results_ccz = results

Compiling: 3 : 10 4     

In [1]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
})

#f, ax = plt.subplots(figsize=(7, 7))


sbs.set_theme()
sbs.set_style("dark")
sbs.set_style("whitegrid", {'grid.linestyle': '-.'})

#locmin = mticker.LogLocator(base=10, subs=np.arange(0.1,1,0.1),numticks=20)  
#ax.yaxis.set_minor_locator(locmin)
#ax.yaxis.set_minor_formatter(mticker.NullFormatter())
#plt.grid(True,which="both")  


marks = ['', '.', 'x', '*']
linestyles = ['-', '--', ':']

dependent = 'volume'
independent = 'n_ctrl'
#fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7, 7))

ax = [
    plt.subplot2grid((1, 5), (0, 0), colspan=2),
    plt.subplot2grid((1, 5), (0, 3), colspan=2)
]

subplot = ax[0]
kwargs = {
'errorbar':('ci', 100)
}

for idx, target in enumerate((1, 2, 3)):
    sbs.lineplot(
        ax=subplot,
        **plt_data(
            results_dag['distillation'],
            results_dag[independent],
            results_dag[dependent],
            target=target
        ),
        label=f'DAG$_{target}$',
        linestyle=linestyles[idx],
        marker='^',
        color=sbs.color_palette()[0],
        **kwargs
    )

    sbs.lineplot(
        ax=subplot,
        **plt_data(
            results_extern['distillation'],
            results_extern[independent],
            results_extern[dependent],
            target=target
        ),
        label=f'Extern$_{target}$',
        linestyle=linestyles[idx],
        marker='s',
        color=sbs.color_palette()[target - 1],
        **kwargs
    )

    obj = sbs.lineplot(
        ax=subplot,
        **plt_data(
            results_ccz['distillation'],
            results_ccz[independent],
            results_ccz[dependent],
            target=target
        ),
        label=f'CCZ$_{target}$',
        linestyle=linestyles[idx],
        marker='v',
        color=sbs.color_palette()[2],
        **kwargs
    )
subplot.legend_.remove()
subplot.yaxis.set_label_position("left")
subplot.yaxis.tick_left()
subplot.set_yscale('log')
subplot.set_ylabel('Space-Time Volume')

dependent = 'runtime'

subplot = ax[1]
for idx, target in enumerate((1, 2, 3)):
    sbs.lineplot(
        ax=subplot,
        **plt_data(
            results_dag['distillation'],
            results_dag[independent],
            results_dag[dependent],
            target=target
        ),
        label=f'DAG$_{target}$',
        linestyle=linestyles[idx],
        marker='^',
        markersize=9,
        color=sbs.color_palette()[0],
        **kwargs
    )

    sbs.lineplot(
        ax=subplot,
        **plt_data(
            results_extern['distillation'],
            results_extern[independent],
            results_extern[dependent],
            target=target
        ),
        label=f'Extern$_{target}$',
        linestyle=linestyles[idx],
        marker='s',
        color=sbs.color_palette()[1],
        **kwargs
    )

    obj = sbs.lineplot(
        ax=subplot,
        **plt_data(
            results_ccz['distillation'],
            results_ccz[independent],
            results_ccz[dependent],
            target=target
        ),
        label=f'CCZ$_{target}$',
        linestyle=linestyles[idx],
        marker='v',
        color=sbs.color_palette()[2],
        **kwargs
    )
    
subplot.legend_.remove()
subplot.yaxis.set_label_position("right")
subplot.yaxis.tick_right()
subplot.set_yscale('log')
subplot.set_ylabel('Cycles')

fig = plt.gcf()
fig.set_size_inches(7, 7)

lines = linestyles * 3
labels = sum(([f'DAG$_{target}$', f'Extern$_{target}$', f'CCZ$_{target}$'] for target in range(3)), start=list())

handles, labels = subplot.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper left', bbox_to_anchor=(0.43, 0.75))
fig.supxlabel('C$^{n}$X')


#sbs.lineplot(x=results_extern['n_ctrl'], y=results_extern['volume'], label="Toffoli Extern")
#sbs.lineplot(x=results_ccz['n_ctrl'], y=results_ccz['volume'], label='Toffoli Factory', **ccz_style, marker='^')


#plt.title('Toffoli Runtimes for Various QCB Sizes')
#plt.xlabel('C$^{n}$X')
#plt.ylabel('Space-Time Volume')

#plt.yscale('log')

#plt.legend()

plt.savefig('multi_toffoli.pdf')

NameError: name 'plt' is not defined